In [1]:
import pandas as pd

In [2]:
# Creación de la dimensión de Tiempo (dim_tiempo)

dataset = '../data/processed/dataset_analitico.csv'
df_analitico = pd.read_csv(dataset, low_memory=False)

# Extraer los periodos únicos (year_month)
dim_tiempo = df_analitico[['year_month']].drop_duplicates().reset_index(drop=True)
dim_tiempo['year_month'] = dim_tiempo['year_month'].astype(str)

# Desglosar el año, mes y calcular el trimestre
dim_tiempo['year'] = dim_tiempo['year_month'].str.split('-').str[0].astype(int)
dim_tiempo['month'] = dim_tiempo['year_month'].str.split('-').str[1].astype(int)
dim_tiempo['quarter'] = dim_tiempo['month'].apply(lambda m: (m - 1) // 3 + 1)

# Asignar un identificador único (time_id)
dim_tiempo.reset_index(inplace=True)
dim_tiempo.rename(columns={'index': 'time_id'}, inplace=True)
dim_tiempo['time_id'] = dim_tiempo['time_id'] + 1  # Para que comience en 1

# Guardar la dimensión de tiempo
dim_tiempo_file = '../data/processed/modeling/dim_tiempo.csv'
dim_tiempo.to_csv(dim_tiempo_file, index=False)


In [3]:
# Creación de la dimensión de Usuarios (dim_usuarios)

users_file = '../data/processed/megaline_users_clean.csv'
dim_usuarios = pd.read_csv(users_file, low_memory=False)

# Guardar la dimensión de usuarios
dim_usuarios_file = '../data/processed/modeling/dim_usuarios.csv'
dim_usuarios.to_csv(dim_usuarios_file, index=False)

In [4]:
# Creación de la dimensión de Planes (dim_planes)

# Cargar la tabla de planes limpia
plans_file = '../data/processed/megaline_plans_clean.csv'
dim_planes = pd.read_csv(plans_file, low_memory=False)
dim_planes.rename(columns={'plan_name': 'plan'}, inplace=True)

# Guardar la dimensión de planes
dim_planes_file = '../data/processed/modeling/dim_planes.csv'
dim_planes.to_csv(dim_planes_file, index=False)


In [ ]:
# Modelamiento con ayuda de claude sonnet 3.7
dataset = '../data/processed/dataset_analitico.csv'
df_analitico = pd.read_csv(dataset, low_memory=False)

# Cargar la dimensión de tiempo para obtener el time_id
dim_tiempo_file = '../data/processed/modeling/dim_tiempo.csv'
dim_tiempo = pd.read_csv(dim_tiempo_file, low_memory=False)

# Unir df_analitico con la dimensión de tiempo a través de 'year_month'
df_fact = df_analitico.merge(dim_tiempo[['time_id', 'year_month']], on='year_month', how='left')

# Renombrar la columna 'plan' a 'plan_id' para el join con la dimensión de planes
df_fact.rename(columns={'plan': 'plan_id'}, inplace=True)

# Seleccionar las columnas que formarán la tabla de hechos
fact_columns = [
    'user_id',      
    'time_id',     
    'plan_id',      
    'total_calls',
    'total_duration',
    'avg_duration',
    'total_mb',
    'sessions_count',
    'total_messages'
]
fact_facturacion = df_fact[fact_columns]

# Guardar la tabla de hechos
fact_facturacion_file = '../data/processed/modeling/fact_facturacion.csv'
fact_facturacion.to_csv(fact_facturacion_file, index=False)


Explicación del Modelo (Star Schema)

Dimensiones:

dim_usuarios: Contiene información del usuario (user_id, first_name, last_name, age, city, reg_date, churn_date, etc.). Se obtiene a partir del archivo megaline_users_clean.csv.
dim_planes: Contiene la información de los planes (plan_id, nombre, costo mensual, minutos, SMS, datos incluidos, etc.), proveniente de megaline_plans_clean.csv. En tu código se renombró la columna de plan.
dim_tiempo: Se creó a partir del campo year_month del dataset analítico, desglosando año, mes y trimestre, y asignándole un identificador único time_id.
Tabla de Hechos (fact_facturación):

Cada registro representa el consumo de un usuario en un periodo determinado (por ejemplo, un mes).
Llaves foráneas:
user_id que relaciona con dim_usuarios.
plan_id que relaciona con dim_planes.
time_id que relaciona con dim_tiempo.
Medidas (Facts):
Consumos agregados: total_calls, total_duration, avg_duration, total_mb, sessions_count, total_messages.

Este modelo en forma de Star Schema permite almacenar la información de forma centralizada en la tabla de hechos, con dimensiones que aportan el contexto (usuarios, planes y tiempo) para realizar consultas y análisis agregados de manera eficiente. Luego, estas relaciones facilitan la preparación del dataset final para entrenar modelos predictivos (por ejemplo, para predecir el costo total que pagará un usuario en el próximo mes).